In [1]:
from moccasin import setup_notebook

setup_notebook()

In [2]:
from moccasin.config import get_active_network

active_network = get_active_network()
print(active_network.name)

mainnet-fork


In [3]:
import boa
from moccasin.config import get_config
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple

TARGET_ALLOCATIONS = {"usdc": 0.3, "weth": 0.7}
STARTING_ETH_BALANCE = int(1000e18)
STARTING_USDC_BALANCE = int(100e6)
STARTING_WETH_BALANCE = int(1e18)

def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)

def _add_token_balance(usdc, weth, active_network):
   
    weth.deposit(value=STARTING_WETH_BALANCE)
    our_address = boa.env.eoa
    
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
        
    usdc.configureMinter(our_address, STARTING_USDC_BALANCE)
    usdc.mint(our_address, STARTING_USDC_BALANCE)
    
    
def setup_script() -> Tuple[ABIContract,ABIContract,ABIContract,ABIContract]:
    print("Starting setup script")
    
    active_network = get_config().get_active_network()
    
    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")

    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc, weth, active_network)

    print(usdc.balanceOf(boa.env.eoa))
 
def moccasin_main():
    setup_script()

moccasin_main()

Starting setup script
100000000
100000000


In [4]:
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")

In [5]:
usdc.balanceOf(boa.env.eoa)
weth.balanceOf(boa.env.eoa)

1000000000000000000

In [6]:
from moccasin.config import get_or_initialize_config

config = get_or_initialize_config()
config.reload()
active_network = config.get_active_network()

aavev3_pool_address_provider = active_network.manifest_named("aavev3_pool_address_provider")
pool_address = aavev3_pool_address_provider.getPool()


print(pool_address)

config.reload()
active_network = config.get_active_network()
pool_contract = active_network.manifest_named("pool", address=pool_address)
print(pool_contract)


0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
<pool interface at 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2>


In [7]:
pool_contract 

<pool interface at 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2>

In [8]:
REFERRAL_CODE = 0   

def deposit(pool_contract, token, amount):
    allowed_amount = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amount < amount:
        token.approve(pool_contract.address, amount)
    print(f"Depositing {amount} into {pool_contract.address}")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFERRAL_CODE)
    


usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)
if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)
if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)
    
(
    totalCollateralBase,
    totalDebtBase,
    availableBorrowsBase,
    currentLiquidationThreshold,
    ltv,
    healthFactor,
) = pool_contract.getUserAccountData(boa.env.eoa)
print(f"""User account data:
        totalCollateralBase: {totalCollateralBase}
        totalDebtBase: {totalDebtBase}
        availableBorrowsBase: {availableBorrowsBase}
        currentLiquidationThreshold: {currentLiquidationThreshold}
        ltv: {ltv}
        healthFactor: {healthFactor}
          """)


Depositing 100000000 into 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
Depositing 1000000000000000000 into 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
User account data:
        totalCollateralBase: 342743021500
        totalDebtBase: 0
        availableBorrowsBase: 275325469171
        currentLiquidationThreshold: 8285
        ltv: 8033
        healthFactor: 115792089237316195423570985008687907853269984665640564039457584007913129639935
          


In [9]:
# goal 30% USDC and 70% WETH
config.reload()
active_network = config.get_active_network()


aavev3_protocol_data_provider = active_network.manifest_named("aavev3_protocol_data_provider")
a_tokens = aavev3_protocol_data_provider.getAllATokens()
print(a_tokens)

[('aEthWETH', Address('0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8')), ('aEthwstETH', Address('0x0B925eD163218f6662a35e0f0371Ac234f9E9371')), ('aEthWBTC', Address('0x5Ee5bf7ae06D1Be5997A1A72006FE6C607eC6DE8')), ('aEthUSDC', Address('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c')), ('aEthDAI', Address('0x018008bfb33d285247A21d44E50697654f754e63')), ('aEthLINK', Address('0x5E8C8A7243651DB1384C0dDfDbE39761E8e7E51a')), ('aEthAAVE', Address('0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9')), ('aEthcbETH', Address('0x977b6fc5dE62598B08C85AC8Cf2b745874E8b78c')), ('aEthUSDT', Address('0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a')), ('aEthrETH', Address('0xCc9EE9483f662091a1de4795249E24aC0aC2630f')), ('aEthLUSD', Address('0x3Fe6a295459FAe07DF8A0ceCC36F37160FE86AA9')), ('aEthCRV', Address('0x7B95Ec873268a6BFC6427e7a28e396Db9D0ebc65')), ('aEthMKR', Address('0x8A458A9dc9048e005d22849F470891b840296619')), ('aEthSNX', Address('0xC7B4c17861357B8ABB91F25581E7263E08DCB59c')), ('aEthBAL', Address('0x2516E7B3F76

In [10]:
for a_token in a_tokens:
    if "WETH" in a_token[0]:
        a_weth = active_network.manifest_named("weth", address=a_token[1])
    if "USDC" in a_token[0]:
        a_usdc = active_network.manifest_named("usdc", address=a_token[1])
print(a_weth)
print(a_usdc)


<weth interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>
<usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>


In [11]:
# Get how much tokens are worth to figure out the portfolio weights

a_usdc_balance = a_usdc.balanceOf(boa.env.eoa)
a_weth_balance = a_weth.balanceOf(boa.env.eoa) 

a_usdc_balance_normalized = a_usdc_balance / 1e6
a_weth_balance_normalized = a_weth_balance / 1e18

print(a_usdc_balance_normalized)
print(a_weth_balance_normalized)


100.0
1.0


In [12]:
config.reload()

def get_price(feed_name: str) -> float:
    active_network = get_active_network()
    price_feed = active_network.manifest_named(feed_name)
    price = price_feed.latestAnswer()
    decimals = price_feed.decimals()
    decimals_normalized = 10**decimals
    return price / decimals_normalized
    
usdc_price = get_price("usdc_usd_price_feed")
weth_price = get_price("eth_usd_price_feed")

print(usdc_price)
print(weth_price)   

1.00000215
3327.43


In [23]:
usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * weth_price
total_value = usdc_value + weth_value

target_usdc_value = 0.3 
target_weth_value = 0.7 

usdc_percent_allocation = usdc_value / total_value
weth_percent_allocation = weth_value / total_value

BUFFER = 0.1

needs_rebalancing = (
    abs(usdc_percent_allocation - target_usdc_value) > BUFFER
    or abs(weth_percent_allocation - target_weth_value) > BUFFER
)

print(needs_rebalancing)

print(target_usdc_value)
print(target_weth_value)


True
0.3
0.7


In [14]:
def calculate_rebalancing_trades(
    usdc_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    weth_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    target_allocations: dict[str, float],  # {"usdc": 0.3, "weth": 0.7}
) -> dict[str, dict]:
    """
    Calculate the trades needed to rebalance a portfolio of USDC and WETH.

    Args:
        usdc_data: Dict containing USDC balance, price and contract
        weth_data: Dict containing WETH balance, price and contract
        target_allocations: Dict of token symbol to target allocation (must sum to 1)

    Returns:
        Dict of token symbol to dict containing contract and trade amount:
            {"usdc": {"contract": Contract, "trade": int},
             "weth": {"contract": Contract, "trade": int}}
    """
    # Calculate current values
    usdc_value = usdc_data["balance"] * usdc_data["price"]
    weth_value = weth_data["balance"] * weth_data["price"]
    total_value = usdc_value + weth_value

    # Calculate target values
    target_usdc_value = total_value * target_allocations["usdc"]
    target_weth_value = total_value * target_allocations["weth"]

    # Calculate trades needed in USD
    usdc_trade_usd = target_usdc_value - usdc_value
    weth_trade_usd = target_weth_value - weth_value

    # Convert to token amounts
    return {
        "usdc": {
            "contract": usdc_data["contract"],
            "trade": usdc_trade_usd / usdc_data["price"],
        },
        "weth": {
            "contract": weth_data["contract"],
            "trade": weth_trade_usd / weth_data["price"],
        },
    }

In [15]:
if needs_rebalancing:
    trades_tokens = calculate_rebalancing_trades(
        {"balance": a_usdc_balance_normalized, "price": usdc_price, "contract": usdc},
        {"balance": a_weth_balance_normalized, "price": weth_price, "contract": weth},
        TARGET_ALLOCATIONS,
    )
    
    trades_tokens["usdc"]["a_token"] = a_usdc
    trades_tokens["weth"]["a_token"] = a_weth
    
    
    
    

In [16]:
trades_token_to_buy = (
    trades_tokens["usdc"]
    if trades_tokens["usdc"]["trade"] > 0
    else trades_tokens["weth"]
)

trades_token_to_sell = (
    trades_tokens["usdc"]
    if trades_tokens["usdc"]["trade"] <= 0
    else trades_tokens["weth"]
)


amount_in = abs(
    int(
        trades_token_to_sell["trade"]
        * (10 ** trades_token_to_sell["contract"].decimals())
    )
)

amount_out = int(
    trades_token_to_buy["trade"]
    * ((10 ** trades_token_to_buy["contract"].decimals()) * 0.95)
)



In [17]:
config.reload()


success = trades_token_to_sell["a_token"].approve(pool_contract.address, amount_in)

withdraw_amount = trades_token_to_sell["a_token"].balanceOf(boa.env.eoa)

pool_contract.withdraw(
    trades_token_to_sell["contract"].address, withdraw_amount, boa.env.eoa
)



1000000000000000000

In [18]:
def swap_exact_input_single(
    swap_router,
    token_in_contract,
    token_out_contract,
    amount_in: int,
    amount_out_min: int,
    pool_fee: int = 3000,  # 0.3% fee tier
) -> int:
    """
    Swaps a fixed amount of token_in for a maximum possible amount of token_out

    Args:
        swap_router: ISwapRouter contract
        token_in_contract: Input token contract
        token_out_contract: Output token contract
        amount_in: Exact amount of input token to swap
        pool_fee: Fee tier (default 0.3% = 3000)

    Returns:
        amount_out: Amount of output token received
    """
    # First approve router to spend token
    token_in_contract.approve(swap_router.address, amount_in)

    # struct ExactInputSingleParams {
    #     address tokenIn;
    #     address tokenOut;
    #     uint24 fee;
    #     address recipient;
    #     uint256 amountIn;
    #     uint256 amountOutMinimum;
    #     uint160 sqrtPriceLimitX96;
    # }
    amount_out = swap_router.exactInputSingle(
        (
            token_in_contract.address,
            token_out_contract.address,
            pool_fee,
            boa.env.eoa,
            int(amount_in),
            int(amount_out_min),
            0,
        )
    )
    return amount_out

In [19]:
config.reload()
active_network = config.get_active_network()
uniswap_swap_router = active_network.manifest_named("uniswap_swap_router")

success = trades_token_to_sell["contract"].approve(uniswap_swap_router.address, amount_in)

print("Time to swap!")
print(
    f"Let's swap {amount_in / (10 ** trades_token_to_sell["contract"].decimals())} {trades_token_to_sell["contract"].symbol()} for at least {amount_out / (10 ** trades_token_to_buy["contract"].decimals())} {trades_token_to_buy["contract"].symbol()}"
)

amount_out = swap_exact_input_single(
            swap_router=uniswap_swap_router,
            token_in_contract=trades_token_to_sell["contract"],
            token_out_contract=trades_token_to_buy["contract"],
            amount_in=amount_in,
            amount_out_min=amount_out,
        )


Time to swap!
Let's swap 0.27896269778778227 WETH for at least 881.815511 USDC


In [20]:
amount = usdc.balanceOf(boa.env.eoa)
deposit(pool_contract, usdc, amount)
amount = weth.balanceOf(boa.env.eoa)
deposit(pool_contract, weth, amount)


Depositing 926969341 into 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
Depositing 721037302212217728 into 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [25]:
a_usdc_balance = a_usdc.balanceOf(boa.env.eoa)
a_weth_balance = a_weth.balanceOf(boa.env.eoa)

a_usdc_balance_normalized = a_usdc_balance / 1e6
a_weth_balance_normalized = a_weth_balance / 1e18

usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * weth_price
total_value = usdc_value + weth_value

target_usdc_value = 0.3 
target_weth_value = 0.7 

usdc_percent_allocation = usdc_value / total_value
weth_percent_allocation = weth_value / total_value



print(usdc_percent_allocation)
print(weth_percent_allocation)
print(a_weth.balanceOf(boa.env.eoa) / 1e18 * weth_price)
print(a_usdc.balanceOf(boa.env.eoa) / 1e6 * usdc_price)



0.2997430774983193
0.7002569225016807
2399.2011504999996
1026.9715489840833
